In [2]:
import sframe                                     # see below for install instruction

In [1]:
import matplotlib.pyplot as plt
import numpy as np
from scipy.sparse import csr_matrix
from sklearn.cluster import KMeans                # we'll be using scikit-learn's KMeans for this assignment
from sklearn.metrics import pairwise_distances
from sklearn.preprocessing import normalize
%matplotlib inline

In [3]:
wiki = sframe.SFrame('./data/people_wiki.gl/')

[INFO] sframe.cython.cy_server: SFrame v1.10 started. Logging /tmp/sframe_server_1468954396.log


In [4]:
def load_sparse_csr(filename):
    loader = np.load(filename)
    data = loader['data']
    indices = loader['indices']
    indptr = loader['indptr']
    shape = loader['shape']
    
    return csr_matrix( (data, indices, indptr), shape)

tf_idf = load_sparse_csr('./data/people_wiki_tf_idf.npz')
map_index_to_word = sframe.SFrame('./data/people_wiki_map_index_to_word.gl/')

### Bipartition the Wikipedia dataset using k-means

- Recall our workflow for clustering text data with k-means:

    - Load the dataframe containing a dataset, such as the Wikipedia text dataset.
    - Extract the data matrix from the dataframe.
    - Run k-means on the data matrix with some value of k.
    - Visualize the clustering results using the centroids, cluster assignments, and the original dataframe. We keep the original dataframe around because the data matrix does not keep auxiliary information (in the case of the text dataset, the title of each article).

- Let us modify the workflow to perform bipartitioning:

    - Load the dataframe containing a dataset, such as the Wikipedia text dataset.
    - Extract the data matrix from the dataframe.
    - Run k-means on the data matrix with k=2.
    - Divide the data matrix into two parts using the cluster assignments.
    - Divide the dataframe into two parts, again using the cluster assignments. This step is necessary to allow for visualization.
    - Visualize the bipartition of data.

Each parent cluster is bipartitioned to produce two child clusters. At the very top is the root cluster, which consists of the entire dataset.

Now we write a wrapper function to bipartition a given cluster using k-means. There are three variables that together comprise the cluster:

- dataframe: a subset of the original dataframe that correspond to member rows of the cluster
- matrix: same set of rows, stored in sparse matrix format
- centroid: the centroid of the cluster (not applicable for the root cluster)
Rather than passing around the three variables separately, we package them into a Python dictionary. The wrapper function takes a single dictionary (representing a parent cluster) and returns two dictionaries (representing the child clusters).

In [5]:
def bipartition(cluster, maxiter=400, num_runs=4, seed=None):
    '''cluster: should be a dictionary containing the following keys
                * dataframe: original dataframe
                * matrix:    same data, in matrix format
                * centroid:  centroid for this particular cluster'''
    
    data_matrix = cluster['matrix']
    dataframe   = cluster['dataframe']
    
    # Run k-means on the data matrix with k=2. We use scikit-learn here to simplify workflow.
    kmeans_model = KMeans(n_clusters=2, max_iter=maxiter, n_init=num_runs, random_state=seed, n_jobs=-1)    
    kmeans_model.fit(data_matrix)
    centroids, cluster_assignment = kmeans_model.cluster_centers_, kmeans_model.labels_
    
    # Divide the data matrix into two parts using the cluster assignments.
    data_matrix_left_child, data_matrix_right_child = data_matrix[cluster_assignment==0], \
                                                      data_matrix[cluster_assignment==1]
    
    # Divide the dataframe into two parts, again using the cluster assignments.
    cluster_assignment_sa = sframe.SArray(cluster_assignment) # minor format conversion
    dataframe_left_child, dataframe_right_child     = dataframe[cluster_assignment_sa==0], \
                                                      dataframe[cluster_assignment_sa==1]
        
    
    # Package relevant variables for the child clusters
    cluster_left_child  = {'matrix': data_matrix_left_child,
                           'dataframe': dataframe_left_child,
                           'centroid': centroids[0]}
    cluster_right_child = {'matrix': data_matrix_right_child,
                           'dataframe': dataframe_right_child,
                           'centroid': centroids[1]}
    
    return (cluster_left_child, cluster_right_child)

The following cell performs bipartitioning of the Wikipedia dataset. Allow 20-60 seconds to finish.

Note. For the purpose of the assignment, we set an explicit seed (seed=1) to produce identical outputs for every run. In pratical applications, you might want to use different random seeds for all runs.

In [6]:
wiki_data = {'matrix': tf_idf, 'dataframe': wiki} # no 'centroid' for the root cluster
left_child, right_child = bipartition(wiki_data, maxiter=100, num_runs=8, seed=1)

Let's examine the contents of one of the two clusters, which we call the left_child, referring to the tree visualization above.

In [7]:
print left_child
print right_child

{'centroid': array([ 0.00022199,  0.00022199,  0.00022199, ...,  0.01077195,
        0.00801833,  0.00199557]), 'matrix': <49491x547979 sparse matrix of type '<type 'numpy.float64'>'
	with 8658969 stored elements in Compressed Sparse Row format>, 'dataframe': Columns:
	URI	str
	name	str
	text	str

Rows: Unknown

Data:
+-------------------------------+---------------------+
|              URI              |         name        |
+-------------------------------+---------------------+
| <http://dbpedia.org/resour... |    Digby Morrell    |
| <http://dbpedia.org/resour... |    Alfred J. Lewy   |
| <http://dbpedia.org/resour... |    Harpdog Brown    |
| <http://dbpedia.org/resour... | Franz Rottensteiner |
| <http://dbpedia.org/resour... |        G-Enka       |
| <http://dbpedia.org/resour... |    Sam Henderson    |
| <http://dbpedia.org/resour... |    Aaron LaCrate    |
| <http://dbpedia.org/resour... |   Trevor Ferguson   |
| <http://dbpedia.org/resour... |     Grant Nelson    |
| <http:

### Visualize the bipartition

We provide you with a modified version of the visualization function from the k-means assignment. For each cluster, we print the top 5 words with highest TF-IDF weights in the centroid and display excerpts for the 8 nearest neighbors of the centroid.

In [8]:
def display_single_tf_idf_cluster(cluster, map_index_to_word):
    '''map_index_to_word: SFrame specifying the mapping betweeen words and column indices'''
    
    wiki_subset   = cluster['dataframe']
    tf_idf_subset = cluster['matrix']
    centroid      = cluster['centroid']
    
    # Print top 5 words with largest TF-IDF weights in the cluster
    idx = centroid.argsort()[::-1]
    for i in xrange(5):
        print('{0:s}:{1:.3f}'.format(map_index_to_word['category'][idx[i]], centroid[idx[i]])),
    print('')
    
    # Compute distances from the centroid to all data points in the cluster.
    distances = pairwise_distances(tf_idf_subset, [centroid], metric='euclidean').flatten()
    # compute nearest neighbors of the centroid within the cluster.
    nearest_neighbors = distances.argsort()
    # For 8 nearest neighbors, print the title as well as first 180 characters of text.
    # Wrap the text at 80-character mark.
    for i in xrange(8):
        text = ' '.join(wiki_subset[nearest_neighbors[i]]['text'].split(None, 25)[0:25])
        print('* {0:50s} {1:.5f}\n  {2:s}\n  {3:s}'.format(wiki_subset[nearest_neighbors[i]]['name'],
              distances[nearest_neighbors[i]], text[:90], text[90:180] if len(text) > 90 else ''))
    print('')

In [9]:
display_single_tf_idf_cluster(left_child, map_index_to_word)
display_single_tf_idf_cluster(right_child, map_index_to_word)

he:1.332 his:0.926 music:0.876 university:0.868 league:0.837 
* Rodney Marsh (footballer)                          43.22699
  rodney william marsh born 11 october 1944 is an english former footballer and football coa
  ch he later worked as a broadcaster he won nine caps
* Adam Haslett                                       43.68907
  adam haslett born december 24 1970 is an american fiction writer he was born in port chest
  er new york and grew up in oxfordshire england
* Tommy Haas                                         43.94744
  thomas mario tommy haas born 3 april 1978 is a german professional tennis player he has co
  mpeted on the atp tour since 1996 after breaking
* Michael Munger                                     44.32072
  michael curtis mike munger born september 23 1958 is an economist and a former chair of th
  e political science department at duke university where he
* Billy Wright (footballer, born 1958)               46.03777
  william billy wright born 28 april 195

Is this hierarchy good enough? When building a hierarchy of clusters, we must keep our particular application in mind. For instance, we might want to build a directory for Wikipedia articles. A good directory would let you quickly narrow down your search to a small set of related articles. The categories of athletes and non-athletes are too general to facilitate efficient search. For this reason, we decide to build another level into our hierarchy of clusters with the goal of getting more specific cluster structure at the lower level. To that end, we subdivide both the athletes and non-athletes clusters.



### Perform recursive bipartitioning

Cluster of athletes. To help identify the clusters we've built so far, let's give them easy-to-read aliases:

In [10]:
athletes = left_child
non_athletes = right_child

In [11]:
# Bipartition the cluster of athletes
left_child_athletes, right_child_athletes = bipartition(athletes, maxiter=100, num_runs=8, seed=1)

The left child cluster mainly consists of baseball players. On the other hand, the right child cluster is a mix of football players and ice hockey players.

In [12]:
display_single_tf_idf_cluster(left_child_athletes, map_index_to_word)
display_single_tf_idf_cluster(right_child_athletes, map_index_to_word)

league:1.700 music:1.659 season:1.652 played:1.487 he:1.373 
* Rodney Marsh (footballer)                          42.86629
  rodney william marsh born 11 october 1944 is an english former footballer and football coa
  ch he later worked as a broadcaster he won nine caps
* Tommy Haas                                         43.76640
  thomas mario tommy haas born 3 april 1978 is a german professional tennis player he has co
  mpeted on the atp tour since 1996 after breaking
* Billy Wright (footballer, born 1958)               45.71184
  william billy wright born 28 april 1958 is an english former professional footballer who p
  layed as a centre half he played 370 games in the
* Adrian Burrows                                     46.19068
  adrian mark burrows born 16 january 1959 is a retired english footballer who played as a c
  entre backhe began his career as an apprentice with
* Dan Siegel (musician)                              46.76403
  dan siegel born in seattle washington is a 

Note. Concerning use of "football"

The occurrences of the word "football" above refer to association football. This sports is also known as "soccer" in United States (to avoid confusion with American football). We will use "football" throughout when discussing topic representation.

Should we keep subdividing the clusters? If so, which cluster should we subdivide? To answer this question, we again think about our application. Since we organize our directory by topics, it would be nice to have topics that are about as coarse as each other. For instance, if one cluster is about baseball, we expect some other clusters about football, basketball, volleyball, and so forth. That is, we would like to achieve similar level of granularity for all clusters.

Notice that the right child cluster is more coarse than the left child cluster. The right cluster possesses a greater variety of topics than the left (ice hockey/football vs. baseball). So the right child cluster should be subdivided further to produce finer child clusters.

Let's give the clusters aliases as well:

In [40]:
baseball            = left_child_athletes
ice_hockey_football = right_child_athletes

Cluster of ice hockey players and football players. In answering the following quiz question, take a look at the topics represented in the top documents (those closest to the centroid), as well as the list of words with highest TF-IDF weights.

In [41]:
# Bipartition the cluster of ice_hockey_football
left_child_athletes, right_child_athletes = bipartition(ice_hockey_football, maxiter=100, num_runs=8, seed=1)

In [42]:
display_single_tf_idf_cluster(left_child_athletes, map_index_to_word)
display_single_tf_idf_cluster(right_child_athletes, map_index_to_word)

university:1.489 research:1.307 he:1.189 art:1.188 professor:1.079 
* Adam Haslett                                       43.47247
  adam haslett born december 24 1970 is an american fiction writer he was born in port chest
  er new york and grew up in oxfordshire england
* Michael Munger                                     43.70049
  michael curtis mike munger born september 23 1958 is an economist and a former chair of th
  e political science department at duke university where he
* Patrick Joyce                                      45.69481
  patrick joyce is a british social historian who has also worked on political history he is
   also known for his theoretical work on the nature
* Howard Alper                                       45.70048
  howard alper oc frsc born october 17 1941 is a canadian chemist he is a professor of chemi
  stry at the university of ottawa he is
* Jon Butler                                         46.74892
  jon butler born 4 june 1940 is a historian a

### Quiz Question. Bipartition the cluster of ice hockey and football players. Which of the two child clusters should be futher subdivided?

Note. To achieve consistent results, use the arguments maxiter=100, num_runs=8, seed=1 when calling the bipartition function.

- The left child cluster
- The right child cluster

Caution. The granularity criteria is an imperfect heuristic and must be taken with a grain of salt. It takes a lot of manual intervention to obtain a good hierarchy of clusters.

If a cluster is highly mixed, the top articles and words may not convey the full picture of the cluster. Thus, we may be misled if we judge the purity of clusters solely by their top documents and words.
Many interesting topics are hidden somewhere inside the clusters but do not appear in the visualization. We may need to subdivide further to discover new topics. For instance, subdividing the ice_hockey_football cluster led to the appearance of golf.

### Quiz Question. Which diagram best describes the hierarchy right after splitting the ice_hockey_football cluster? Refer to the quiz form for the diagrams.

Cluster of non-athletes. Now let us subdivide the cluster of non-athletes.

In [ ]:
# Bipartition the cluster of non-athletes
left_child_non_athletes, right_child_non_athletes = bipartition(non_athletes, maxiter=100, num_runs=8, seed=1)

display_single_tf_idf_cluster(left_child_non_athletes, map_index_to_word)
display_single_tf_idf_cluster(right_child_non_athletes, map_index_to_word)

The first cluster consists of scholars, politicians, and government officials whereas the second consists of musicians, artists, and actors. Run the following code cell to make convenient aliases for the clusters.

In [ ]:
scholars_politicians_etc = left_child_non_athletes
musicians_artists_etc = right_child_non_athletes

Quiz Question. Let us bipartition the clusters scholars_politicians_etc and musicians_artists_etc. Which diagram best describes the resulting hierarchy of clusters for the non-athletes? Refer to the quiz for the diagrams.

Note. Use maxiter=100, num_runs=8, seed=1 for consistency of output.